In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import seaborn.objects as so
from sklearn.linear_model import LogisticRegression



from src.DataProcessor import DataProcessor
# TODO: create new env for loans, and isntall src

In [17]:
PATH = r'C:\Users\mds8301\Documents\Github\credit_modeling\data\loans_2007.csv'
DATA = (DataProcessor(PATH, target = 'loan_status')
        .load_data()
        .clean_data()
        .processes_dummies()
        .extract_feature_names()
        )

0        1
1        0
2        1
3        1
5        1
        ..
39631    1
39642    1
39692    0
39735    0
39749    1
Name: loan_status, Length: 36990, dtype: int64